In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
#importing libs
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB #this is for the model type
from sklearn import metrics
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer

In [28]:
#reading the data
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Studily/Dataset_MCnew.csv")
#print(dataset.columns.tolist())
convert_dict = {'id': str,
                'title': str,
                'Category': int
                }
dataset = dataset.astype(convert_dict)
X_train_non, X_test_non, y_train, y_test = train_test_split(dataset['title'], dataset['Category'])
X_train_non = np.array(X_train_non)
X_test_non = np.array(X_test_non)
y_train = np.array(y_train)
y_test = np.array(y_test)
#X_train_non.to_csv('bye.csv')
#dataset

In [29]:
dataset['Category'].value_counts()

0    893
4    893
3    892
1    890
2    889
Name: Category, dtype: int64

In [30]:
#initialising stopwords and lematization
stopword_list = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [31]:
X_train, X_test = [], []
#pre-processing the title
for i in range(0, len(X_train_non)):
  #print(X_train_non[i])
  _title = re.sub(r'[^a-zA-Z]', ' ', X_train_non[i])
  _title = _title.lower()
  _title = _title.split()
  _title = [lemmatizer.lemmatize(word) for word in _title if not word in set(stopword_list)]
  _title = ' '.join(_title)
  X_train.append(_title)

for i in range(0, len(X_test_non)):
  _title = re.sub(r'[^a-zA-Z]', ' ', X_test_non[i])
  _title = _title.lower()
  _title = _title.split()
  _title = [lemmatizer.lemmatize(word) for word in _title if not word in set(stopword_list)]
  _title = ' '.join(_title)
  X_test.append(_title)

In [32]:
y_test[9] #checking values of train test

2

In [33]:
#initialising tfidf vectorizer
tf_idf = TfidfVectorizer()
tfidf_train = tf_idf.fit_transform(X_train)
tfidf_test = tf_idf.transform(X_test)

In [34]:
#naive bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(tfidf_train, y_train)

#predicted y
y_pred = naive_bayes_classifier.predict(tfidf_test)

print(metrics.classification_report(y_test, y_pred, target_names=['Educational', 'Music','Sports','Gaming','Movies']))

              precision    recall  f1-score   support

 Educational       0.96      0.96      0.96       225
       Music       0.94      0.98      0.96       232
      Sports       0.95      0.92      0.94       207
      Gaming       0.96      0.95      0.96       221
      Movies       0.97      0.96      0.96       230

    accuracy                           0.96      1115
   macro avg       0.96      0.95      0.95      1115
weighted avg       0.96      0.96      0.96      1115



In [48]:
import pickle
filename = '/content/drive/MyDrive/Colab Notebooks/Studily/Models/Bayes_Multi_Data_model.sav'
pickle.dump(naive_bayes_classifier, open(filename, 'wb'))

#This next part is for doing test prediction

In [45]:
#test prediction
test_data = ['brand new shows to watch while playing games and doing maths and JEE and NEET']
_test = re.sub(r'[^a-zA-Z]', ' ', test_data[0])
_test = _test.lower()
_test = _test.split()
_test = [lemmatizer.lemmatize(word) for word in _test if not word in set(stopword_list)]
text_processed = [' '.join(_test)]
print(text_processed)

['brand new show watch playing game math jee neet']


In [46]:
test_input = tf_idf.transform(text_processed)
test_input.shape

(1, 5778)

In [49]:
result_probs = naive_bayes_classifier._predict_proba_lr(test_input[0])
result = naive_bayes_classifier.predict(test_input[0])

if result_probs[0][result][0]>=0.5:
  if result == 0:
    print("Educational", result_probs[0][result][0]*100,"% confidence")
  elif result == 1:
    print("Music", result_probs[0][result][0]*100,"% confidence")
  elif result == 2:
    print("Sports", result_probs[0][result][0]*100,"% confidence")
  elif result == 3:
    print("Gaming", result_probs[0][result][0]*100,"% confidence")
  elif result == 4:
    print("Movies", result_probs[0][result][0]*100,"% confidence")
else:
  print("Unable to classify")
  print(result, result_probs[0][result][0])

AttributeError: ignored